In [5]:
import os, shutil, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.image as mpimg
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Activation, BatchNormalization
from keras.regularizers import l2
from sklearn.utils import class_weight 
from keras.applications import vgg16

In [6]:
train_dir = 'Human_Face_Regonition_Images\\split2\\train'
validation_dir = 'Human_Face_Regonition_Images\\split2\\validation'
test_dir = 'Human_Face_Regonition_Images\\split2\\test'

In [7]:
train_data_gen_aug = ImageDataGenerator(rescale=1./255, 
                                        rotation_range=30, 
                                        width_shift_range=.2, 
                                        height_shift_range=.2,
                                        zoom_range=0.2)
train_generator_aug = train_data_gen_aug.flow_from_directory(train_dir, target_size=(256, 256), batch_size=128)

validation_data_gen_aug = ImageDataGenerator(rescale=1./255, 
                                        rotation_range=30, 
                                        width_shift_range=.2, 
                                        height_shift_range=.2,
                                        zoom_range=0.2)
validation_generator_aug = validation_data_gen_aug.flow_from_directory(validation_dir, target_size=(256, 256), batch_size=128)

test_data_gen_aug = ImageDataGenerator(rescale=1./255, 
                                        rotation_range=30, 
                                        width_shift_range=.2, 
                                        height_shift_range=.2,
                                        zoom_range=0.2)
test_generator_aug = test_data_gen_aug.flow_from_directory(test_dir, target_size=(256, 256), batch_size=128)

Found 16000 images belonging to 6 classes.
Found 5000 images belonging to 6 classes.
Found 3108 images belonging to 6 classes.


In [8]:
weight = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_generator_aug.classes), 
            train_generator_aug.classes)

train_class_weights = dict(enumerate(weight))
train_class_weights

C:\Users\soohy\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2 3 4 5], y=[0 0 0 ... 5 5 5] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


{0: 1.1185682326621924,
 1: 4.294149221685454,
 2: 5.698005698005698,
 3: 0.33146882121400456,
 4: 0.9399600516978028,
 5: 1.6210739614994933}

In [9]:
cnn_base = vgg16.VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(256, 256, 3))

model = models.Sequential()
model.add(cnn_base)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='softmax'))

# cnn_base.trainable = False

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

result = model.fit(train_generator_aug,
                    steps_per_epoch=40,
                    epochs=15,
                    batch_size=64,
                    class_weight=train_class_weights,
                    validation_data=(validation_generator_aug),
                    validation_steps=25)

Epoch 1/15
